In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from pyvis.network import Network

In [2]:
fname = "HoursJanMay2022.xlsx"
raw_df = pd.read_excel(fname)
df = raw_df.groupby(['Employee','Project','Hour or cost type']).sum()['Quantity']

In [3]:

ignore_proj = ['Other personnel related costs', 'Work council', 'Acquisition', 
               'Call Activities', 'Communication','Knowledge Development', 'Operations & Support',
               'Line management and internal strategy','Parental Leave','Personal development (not project related)','Dissemination & Community']
ignore_emp = ['Nicolas Renaud', 'Yifat Dzigan', 'Willem van Hage','Rob van Nieuwpoort',
              'Rena Bakhshi','Pablo Lopez-Tarifa','Niels Drost','Lars Ridder','Jisk Attema','Patrick Bos',
              'Maria Chertova', 'Jesús Garcia González']

In [5]:
G0 = nx.Graph()

for (emp, proj, _), hours in df.items():
    
    if proj not in ignore_proj:
        if emp not in ignore_emp:
            G0.add_node(emp, type='rse')
            G0.add_node(proj, type='project')
            G0.add_edge(emp,proj,weight=hours)

In [17]:
G = nx.Graph()
for n,attr in G0.nodes.data():

    if attr['type']=='project':
        e = list(G0.edges(n))
        nedge = len(e)
        for ie in range(nedge-1):
            eng1 = e[ie][1]
            for iie in range(1,nedge):
                eng2 = e[iie][1]

                G.add_node(eng1)
                G.add_node(eng2)
                
                w1 = G0.get_edge_data(eng1,n)['weight']
                w2 = G0.get_edge_data(eng2,n)['weight']
                w = w1+w2
                if w>2E2:
                    w = np.exp(0.001*w)-1
                    G.add_edge(eng1,eng2,weight=w

In [18]:

g = Network(height='800px', width='100%',heading='',bgcolor='black',font_color="white", notebook=True)
# g.barnes_hut()
g.from_nx(G)

g.toggle_physics(True)
g.show_buttons(filter_=['physics'])

g.show('graph.html')